In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor as RFF
import xgboost as xgb

from pprint import pprint
import math

from scipy.stats import kurtosis, skew

In [2]:
from datetime import datetime

In [3]:
df_raw = pd.read_csv('../data/data_converted.csv',nrows=None,
                    dtype={'fullVisitorId': str},
                    parse_dates=['date'])

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 40 columns):
channelGrouping               903653 non-null int64
date                          903653 non-null datetime64[ns]
fullVisitorId                 903653 non-null object
sessionId                     903653 non-null int64
visitId                       903653 non-null int64
visitNumber                   903653 non-null int64
visitStartTime                903653 non-null int64
device_browser                903653 non-null int64
device_deviceCategory         903653 non-null int64
device_isMobile               903653 non-null bool
device_operatingSystem        903653 non-null int64
geoNetwork_city               903653 non-null int64
geoNetwork_continent          903653 non-null int64
geoNetwork_country            903653 non-null int64
geoNetwork_metro              903653 non-null int64
geoNetwork_networkDomain      903653 non-null int64
geoNetwork_region             903653 non-null in

In [5]:
df_raw.shape

(903653, 40)

In [6]:
lb = LabelEncoder()
df_raw['fullVisitorId']  = lb.fit_transform(df_raw['fullVisitorId'])

In [7]:
df_raw = df_raw[700000:]

In [8]:
df_train = df_raw[df_raw['date'] < datetime.strptime('2017-7-1','%Y-%m-%d')]
df_valid = df_raw[df_raw['date'] >= datetime.strptime('2017-7-1','%Y-%m-%d')]
del df_train['date']
del df_valid['date']

In [9]:
df_train.shape,df_valid.shape

((174412, 39), (29241, 39))

In [10]:
X_train ,y_train = df_train.drop('totals_transactionRevenue',axis=1),df_train['totals_transactionRevenue']
X_valid ,y_valid = df_valid.drop('totals_transactionRevenue',axis=1),df_valid['totals_transactionRevenue']

In [15]:
# params = {
#     'max_depth':10,
#     'learning_rate':.01,
#     'n_estimators':100,
#     'objective':'reg:linear',
#     'n_jobs':-1,
#     'random_state':1,
# }



In [12]:
def run():
    params = {'objective': 'reg:linear',
          'learning_rate':.01,
          'eval_metric': 'rmse',
          'eta': 0.001,
          'max_depth': 10,
          'subsample': 0.6,
          'colsample_bytree': 0.6,
          'alpha':0.001,
          'random_state': 42,
          'silent': True,
         }


    xgb_train_data = xgb.DMatrix(X_train, y_train)
    xgb_val_data = xgb.DMatrix(X_valid, y_valid)
    
    model = xgb.train(params, xgb_train_data, 
          evals= [(xgb_train_data, 'train'), (xgb_val_data, 'valid')],
          early_stopping_rounds=100, 
          verbose_eval=20
         )


In [17]:

params = {'objective': 'reg:linear',
          'learning_rate':.01,
          'eval_metric': 'rmse',
          'eta': 0.001,
          'max_depth': 10,
          'subsample': 0.6,
          'colsample_bytree': 0.6,
          'alpha':0.001,
          'random_state': 42,
          'silent': True,
          'n_jobs':-1
         }


xgb_train_data = xgb.DMatrix(X_train, y_train)
xgb_val_data = xgb.DMatrix(X_valid, y_valid)



model = xgb.train(params, xgb_train_data, 
          evals= [(xgb_train_data, 'train'), (xgb_val_data, 'valid')],
          early_stopping_rounds=100, 
          verbose_eval=20
         )

[0]	train-rmse:1.95989	valid-rmse:2.18903
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[9]	train-rmse:1.897	valid-rmse:2.14144
